# Feature 3.1 — Exploratory Data Analysis (EDA)
This notebook implements the acceptance criteria and tasks for Feature 3.1 (EDA): profiling, churn definition (>90 days inactivity), CLV draft (12-month net margin), leakage guardrails, non-leaky splits, naive baselines, MLflow logging, and drift/overlap checks between brands (Contoso vs EuroStyle).

Notes:
- Uses a synthetic fallback dataset so the notebook runs end-to-end without data access.
- Replace placeholders with your actual table names/paths and set `USE_SYNTHETIC = False` when connecting to real data.

In [ ]:
# Imports and setup
import os, sys, math, json, warnings, random
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.model_selection import GroupKFold
try:
    import mlflow
except Exception as e:
    mlflow = None

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100)
sns.set(style='whitegrid')

## Configuration
- Freeze seed for reproducibility.
- Record dataset snapshot and churn horizon.
- Set MLflow experiment (optional).

In [ ]:
# Config
SEED = 42
random.seed(SEED); np.random.seed(SEED)
DATASET_SNAPSHOT = os.environ.get('DATASET_SNAPSHOT', datetime.utcnow().strftime('%Y-%m-%d'))
CHURN_HORIZON_DAYS = int(os.environ.get('CHURN_HORIZON_DAYS', '90'))
CUTOFF_DATE = pd.to_datetime(os.environ.get('CUTOFF_DATE', '2024-12-31'))
USE_SYNTHETIC = os.environ.get('USE_SYNTHETIC', 'true').lower() in ('1','true','yes')

# MLflow setup (optional)
MLFLOW_EXPERIMENT = os.environ.get('MLFLOW_EXPERIMENT', '/Feature_3_1_EDA')
if mlflow is not None:
    try:
        mlflow.set_experiment(MLFLOW_EXPERIMENT)
    except Exception as e:
        print('MLflow not configured:', e)

print({'seed': SEED, 'dataset_snapshot': DATASET_SNAPSHOT, 'churn_horizon_days': CHURN_HORIZON_DAYS, 'cutoff_date': str(CUTOFF_DATE), 'use_synthetic': USE_SYNTHETIC})

## Data loading
Replace placeholders with real tables/paths. Expected columns (minimum):
- customers: customer_id, brand
- transactions: customer_id, tx_date, amount, cost, brand
If `USE_SYNTHETIC=True`, a small dataset will be generated.

In [ ]:
def make_synthetic(n_customers=1000, start='2023-01-01', end='2024-12-31'):
    rng = np.random.default_rng(SEED)
    customers = pd.DataFrame({
        'customer_id': np.arange(1, n_customers+1),
        'brand': rng.choice(['Contoso','EuroStyle'], size=n_customers, p=[0.6,0.4])
    })
    dates = pd.date_range(start=start, end=end, freq='D')
    rows = []
    for cid, brand in customers[['customer_id','brand']].itertuples(index=False):
        k = rng.poisson(10)
        if k == 0: continue
        tx_days = rng.choice(dates, size=k, replace=False)
        tx_days.sort()
        for d in tx_days:
            amount = float(np.round(rng.normal(60 if brand=='Contoso' else 55, 20), 2))
            amount = max(5.0, amount)
            cost = float(np.round(amount * rng.uniform(0.5, 0.8), 2))
            rows.append((cid, pd.Timestamp(d), amount, cost, brand))
    tx = pd.DataFrame(rows, columns=['customer_id','tx_date','amount','cost','brand'])
    return customers, tx

if USE_SYNTHETIC:
    customers_df, tx_df = make_synthetic()
else:
    # TODO: Replace with actual load, e.g., Spark or pandas from Lakehouse/Delta
    # from pyspark.sql import SparkSession
    # spark = SparkSession.builder.getOrCreate()
    # tx_df = spark.table('bronze.transactions').toPandas()
    # customers_df = spark.table('bronze.customers').toPandas()
    raise RuntimeError('Set USE_SYNTHETIC=True or implement real data load')

customers_df.head(), tx_df.head()

## Quick profiling
Shapes, missingness, distributions, outliers, correlations.

In [ ]:
print('customers_df shape:', customers_df.shape)
print('tx_df shape:', tx_df.shape)
display(customers_df.head(3))
display(tx_df.head(3))

# Missingness
missing_tx = tx_df.isna().mean().sort_values(ascending=False)
missing_cu = customers_df.isna().mean().sort_values(ascending=False)
print('Missing (transactions): ', missing_tx)
print('Missing (customers): ', missing_cu)

# Distributions
fig, axes = plt.subplots(1, 2, figsize=(12,4))
sns.histplot(tx_df['amount'], ax=axes[0], kde=True)
axes[0].set_title('Amount distribution')
tx_per_cust = tx_df.groupby('customer_id').size()
sns.histplot(tx_per_cust, ax=axes[1], kde=False)
axes[1].set_title('Transactions per customer')
plt.tight_layout(); plt.show()

# Outliers via IQR
q1, q3 = tx_df['amount'].quantile([0.25, 0.75])
iqr = q3 - q1
upper = q3 + 1.5*iqr
outlier_rate = (tx_df['amount'] > upper).mean()
print({'amount_outlier_rate': float(outlier_rate), 'upper_whisker': float(upper)})

# Correlation
num_cols = tx_df.select_dtypes(include=[np.number]).columns
if len(num_cols) > 1:
    plt.figure(figsize=(6,4))
    sns.heatmap(tx_df[num_cols].corr(), annot=False, cmap='Blues')
    plt.title('Numeric correlation (transactions)'); plt.show()

## Churn label (> 90 days inactivity) and prevalence
Compute last activity per customer relative to CUTOFF_DATE.

In [ ]:
tx_df['tx_date'] = pd.to_datetime(tx_df['tx_date'])
last_tx = tx_df.groupby('customer_id')['tx_date'].max().rename('last_activity_date')
cust = customers_df.merge(last_tx, on='customer_id', how='left')
cust['days_inactive'] = (CUTOFF_DATE - cust['last_activity_date']).dt.days
cust['churn_90d'] = cust['days_inactive'] > CHURN_HORIZON_DAYS

prevalence = cust['churn_90d'].mean()
prev_by_brand = cust.groupby('brand')['churn_90d'].mean().sort_values(ascending=False)
print({'churn_prevalence_overall': float(prevalence)})
print('churn_prevalence_by_brand:
', prev_by_brand)
cust.head()

## CLV (12-month net margin) — draft
Draft a simple CLV proxy: past 12-month net margin as a baseline (can be refined later).

In [ ]:
horizon_start = CUTOFF_DATE - pd.Timedelta(days=365)
tx_12m = tx_df[(tx_df['tx_date'] > horizon_start) & (tx_df['tx_date'] <= CUTOFF_DATE)].copy()
tx_12m['margin'] = tx_12m['amount'] - tx_12m['cost']
clv = tx_12m.groupby('customer_id')['margin'].sum().rename('clv_12m_margin')
cust = cust.merge(clv, on='customer_id', how='left').fillna({'clv_12m_margin': 0.0})
cust[['customer_id','brand','clv_12m_margin','churn_90d']].head()

## Leakage checklist
Flag fields with potential future information relative to CUTOFF_DATE.

In [ ]:
potential_leak_cols = [c for c in tx_df.columns if 'cancel' in c.lower() or 'refund' in c.lower() or 'return' in c.lower()]
date_cols = [c for c in tx_df.columns if 'date' in c.lower()]
post_cutoff_flags = {}
for c in date_cols:
    try:
        post_cutoff_flags[c] = bool((pd.to_datetime(tx_df[c]) > CUTOFF_DATE).any())
    except Exception:
        post_cutoff_flags[c] = False
print({'leakage_keywords': potential_leak_cols, 'post_cutoff_date_cols': post_cutoff_flags})

## Non‑leaky splits
Use a time-based holdout and, optionally, GroupKFold by customer. Persist artifacts.

In [ ]:
# Time-based split: train until CUTOFF_DATE - 90d, validate next 45d, test next 45d (example)
train_end = CUTOFF_DATE - pd.Timedelta(days=90)
val_end = train_end + pd.Timedelta(days=45)

cust_dates = tx_df.groupby('customer_id')['tx_date'].max().rename('last_tx')
cust_split = customers_df[['customer_id','brand']].merge(cust_dates, on='customer_id', how='left')
cust_split['split'] = np.where(cust_split['last_tx'] <= train_end, 'train',
                           np.where(cust_split['last_tx'] <= val_end, 'val', 'test'))
cust_split['split'].value_counts(dropna=False)

# Persist artifacts
os.makedirs('artifacts/splits', exist_ok=True)
cust_split[['customer_id','split']].to_csv('artifacts/splits/feature_3_1_splits.csv', index=False)
with open('artifacts/splits/feature_3_1_split_meta.json','w') as f:
    json.dump({'seed': SEED, 'cutoff_date': str(CUTOFF_DATE), 'train_end': str(train_end), 'val_end': str(val_end)}, f, indent=2)
cust_split.head()

## Baselines: Rule-based churn and RFM
Compute simple yardsticks and log to MLflow.

In [ ]:
# Rule-based churn baseline: predict churn if days_inactive > THRESH
THRESH = CHURN_HORIZON_DAYS
pred_score = (cust['days_inactive'] / (THRESH + 1)).clip(0, 1.5)
y_true = cust['churn_90d'].astype(int)
try:
    auc = roc_auc_score(y_true, pred_score)
    ap = average_precision_score(y_true, pred_score)
except Exception:
    auc = float('nan'); ap = float('nan')
print({'rule_auc': float(auc), 'rule_pr_auc': float(ap)})

# RFM bins (simple)
recency = (CUTOFF_DATE - cust['last_activity_date']).dt.days.fillna(1e9)
frequency = tx_df.groupby('customer_id').size().reindex(cust['customer_id']).fillna(0)
monetary = tx_df.groupby('customer_id')['amount'].sum().reindex(cust['customer_id']).fillna(0)
cust['R_bin'] = pd.qcut(recency, q=3, labels=['R3','R2','R1'])
cust['F_bin'] = pd.qcut(frequency.rank(method='first'), q=3, labels=['F1','F2','F3'])
cust['M_bin'] = pd.qcut(monetary.rank(method='first'), q=3, labels=['M1','M2','M3'])
cust['RFM'] = cust['R_bin'].astype(str) + cust['F_bin'].astype(str) + cust['M_bin'].astype(str)
rfm_dist = cust['RFM'].value_counts().head(10)
print('Top RFM bins:', rfm_dist)

# Log to MLflow
if mlflow is not None:
    try:
        with mlflow.start_run(run_name='feature_3_1_baselines'):
            mlflow.log_params({'seed': SEED, 'churn_horizon_days': CHURN_HORIZON_DAYS, 'cutoff_date': str(CUTOFF_DATE)})
            mlflow.log_metrics({'rule_auc': float(auc), 'rule_pr_auc': float(ap)})
            # Save small artifacts
            os.makedirs('artifacts/baselines', exist_ok=True)
            rfm_path = 'artifacts/baselines/rfm_top10.csv'
            rfm_dist.to_csv(rfm_path)
            mlflow.log_artifact(rfm_path)
    except Exception as e:
        print('MLflow logging skipped:', e)

## Drift and brand overlaps (EuroStyle vs Contoso)
Compare feature distributions across brands using KS test or Population Stability Index (PSI).

In [ ]:
from scipy.stats import ks_2samp

def psi(expected, actual, buckets=10, eps=1e-6):
    qs = np.linspace(0, 1, buckets+1)
    e_bins = np.quantile(expected, qs)
    a_bins = e_bins  # use expected bins
    e_counts, _ = np.histogram(expected, bins=e_bins)
    a_counts, _ = np.histogram(actual, bins=a_bins)
    e_pct = e_counts / (e_counts.sum() + eps)
    a_pct = a_counts / (a_counts.sum() + eps)
    val = ((a_pct - e_pct) * np.log((a_pct + eps) / (e_pct + eps))).sum()
    return float(val)

numerics = ['amount']
res = []
for col in numerics:
    a = tx_df.loc[tx_df['brand']=='Contoso', col].dropna()
    b = tx_df.loc[tx_df['brand']=='EuroStyle', col].dropna()
    if len(a) > 10 and len(b) > 10:
        ks = ks_2samp(a, b).statistic
        psi_val = psi(a.values, b.values)
        res.append({'feature': col, 'ks': float(ks), 'psi': float(psi_val)})
pd.DataFrame(res)

## Save EDA artifacts and summary
Outputs: split artifacts, top issues (placeholder), and basic README text for the readout.

In [ ]:
os.makedirs('artifacts/eda', exist_ok=True)
# Example top issues placeholder (replace during real EDA)
top_issues = [
    {'issue': 'Check missing last_activity_date for some customers', 'severity': 'medium', 'owner': 'DE'},
    {'issue': 'Outliers in amount distribution (upper whisker exceeded)', 'severity': 'low', 'owner': 'DA'},
    {'issue': 'Clarify cost vs margin fields semantics', 'severity': 'high', 'owner': 'DA'}
]
pd.DataFrame(top_issues).to_csv('artifacts/eda/top_issues.csv', index=False)
with open('artifacts/eda/README_feature_3_1_summary.txt','w') as f:
    f.write(f"Dataset snapshot: {DATASET_SNAPSHOT}\nCutoff date: {CUTOFF_DATE.date()}\nChurn horizon: {CHURN_HORIZON_DAYS} days\nBaseline AUC (rule): {auc:.3f} | PR-AUC: {ap:.3f}\n\nSee artifacts/splits and artifacts/baselines.\n")
print('Saved artifacts under artifacts/eda and artifacts/splits/baselines')